In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import joblib

In [2]:
# Load Data
df = pd.read_excel('data/emotion.xlsx')


In [4]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
texts = df['text'].astype(str).values
labels = df['emotion'].astype(str).values

In [6]:
# Encode Labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(set(labels_encoded))

In [7]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, padding='post', maxlen=50)

In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(padded, labels_encoded, test_size=0.2, random_state=42)

In [20]:
# Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=32, input_shape=(50,)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


d:\SDE+ML\Real-Time Emotion Detection Chat App\emotion_model\venv\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 50, 32)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321,254 (1.23 MB)

 Trainable params: 321,254 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train Model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3365 - loss: 1.6187 - val_accuracy: 0.4172 - val_loss: 1.5731
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4056 - loss: 1.5360 - val_accuracy: 0.5200 - val_loss: 1.4514
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5481 - loss: 1.3438 - val_accuracy: 0.5347 - val_loss: 1.2383
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6096 - loss: 1.0452 - val_accuracy: 0.6650 - val_loss: 0.9525
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7580 - loss: 0.7836 - val_accuracy: 0.7709 - val_loss: 0.7727
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8381 - loss: 0.5547 - val_accuracy: 0.8012 - val_loss: 0.6322
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8704 - loss: 0.4110 - val_accuracy: 0.8100 - val_loss: 0.5332
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9074 - loss: 0.3046 - val_accuracy: 0.

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8500 - loss: 0.4411
Test Accuracy: 0.8500000238418579


In [24]:

# Example Prediction
test_text = ["I am damn"]


sequence = tokenizer.texts_to_sequences(test_text)
padded_sequence = pad_sequences(sequence, maxlen=50, padding='post')

prediction = model.predict(padded_sequence)
predicted_label_index = prediction.argmax(axis=1)[0]
predicted_emotion = label_encoder.inverse_transform([predicted_label_index])[0]

print(f"Predicted emotion: {predicted_emotion}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predicted emotion: joy


In [41]:
test_text = [
    "I hate when people lie to me.",
    "Nothing excites me anymore.",
    "I'm beyond thrilled with this achievement!",
    "This is just boring and pointless.",
    "I'm feeling anxious and restless today.",
    "Such a peaceful and calm evening.",
    "Why is everyone so rude these days?",
    "I feel completely abandoned and sad.",
    "Life feels so fulfilling right now.",
    "I'm nervous but a little excited too."
]
sequences = tokenizer.texts_to_sequences(test_text)
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

predictions = model.predict(padded_sequences)
predicted_classes = predictions.argmax(axis=1)
predicted_emotions = label_encoder.inverse_transform(predicted_classes)

for i, text in enumerate(test_text):
    print(f"Text: {text}")
    print(f"Predicted Emotion: {predicted_emotions[i]}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Text: I hate when people lie to me.
Predicted Emotion: joy
--------------------------------------------------
Text: Nothing excites me anymore.
Predicted Emotion: sadness
--------------------------------------------------
Text: I'm beyond thrilled with this achievement!
Predicted Emotion: joy
--------------------------------------------------
Text: This is just boring and pointless.
Predicted Emotion: sadness
--------------------------------------------------
Text: I'm feeling anxious and restless today.
Predicted Emotion: fear
--------------------------------------------------
Text: Such a peaceful and calm evening.
Predicted Emotion: joy
--------------------------------------------------
Text: Why is everyone so rude these days?
Predicted Emotion: anger
--------------------------------------------------
Text: I feel completely abandoned and sad.
Predicted Emotion: sadness
--------------------------------------------------
Text: Life feels so fulf

In [45]:
model.save("saved_model/emotion_model.keras") 

In [46]:
# Tokenizer and Encoder via joblib
joblib.dump(tokenizer, "saved_model/tokenizer.joblib")
joblib.dump(label_encoder, "saved_model/label_encoder.joblib")

['saved_model/label_encoder.joblib']